In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
temp_data=pd.read_csv('IMDB Dataset.csv')
df=temp_data[:10000]

In [ ]:
df['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

In [ ]:
df.duplicated().sum()

17

In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
import re
def remove_tags(raw_text):
  cleaned_text=re.sub(re.compile("<.*?>"),'',raw_text)
  return cleaned_text

In [ ]:
df['review']=df['review'].apply(remove_tags)

<ipython-input-178-16f7647403ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_tags)


In [ ]:
df['review'][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [ ]:
df['review']=df['review'].apply(lambda x:x.lower())

<ipython-input-180-abab709f0c77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:x.lower())


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

# Define a function to remove stopwords from a list of words
def remove_stopwords(word_list):
    return [word for word in word_list if word not in sw_list]

# Apply the function to each row in the 'review' column
df['review'] = df['review'].str.split().apply(remove_stopwords).str.join(' ')


<ipython-input-182-c0e5bbd37d29>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.split().apply(remove_stopwords).str.join(' ')


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
from gensim.models import Word2Vec
model=Word2Vec(
    window=10,
    min_count=5
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=5)

(5693948, 6222070)

In [ ]:
def document_vector(doc):
  doc=[word for word in doc.split() if word in model.wv.index_to_key]
  return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['review'][0])

array([-0.00767279,  0.36776376,  0.10511692,  0.1396755 ,  0.00449843,
       -0.44263119,  0.14803712,  0.66219246, -0.23662291, -0.20628081,
       -0.1647654 , -0.32552025,  0.00543609, -0.04349521,  0.1355837 ,
       -0.15745465, -0.02249509, -0.21562083, -0.0482107 , -0.5759054 ,
        0.17671582,  0.14823067,  0.14473967, -0.329305  , -0.1395143 ,
       -0.14332479, -0.21636066, -0.03023511, -0.36705318, -0.09396933,
        0.1986634 , -0.16993853,  0.14775263, -0.20582455, -0.08968441,
        0.32421637,  0.13907391, -0.3901842 , -0.29742765, -0.59239715,
        0.19931488, -0.15592562, -0.12406887, -0.09953684,  0.3315545 ,
        0.01454935, -0.090537  , -0.22288215,  0.07736766,  0.23682798,
        0.01630309, -0.21542937, -0.29600403, -0.14904267,  0.06676942,
        0.10042451,  0.1602676 ,  0.1620983 , -0.22224954,  0.08281311,
        0.07153685,  0.08584232,  0.00850402, -0.06248961, -0.22382855,
        0.20669223,  0.14905761,  0.19083771, -0.38868406,  0.40

In [ ]:
from tqdm import tqdm
X=[]
for doc in tqdm(df['review'].values):
  X.append(document_vector(doc))


100%|██████████| 10000/10000 [04:49<00:00, 34.53it/s]


In [ ]:
X=np.array(X)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7845